# Import Libraries

In [1]:
import socket
import pickle
import numpy as np
import pandas as pd
import json 
import warnings
warnings.filterwarnings('ignore')

# Send And Receive Func for Huge Data Size

In [2]:
# Receive data until fully received (For Huge data => chunk by chunk)
def receive_data(socket, length):
    data = b''
    while len(data) < length:
        packet = socket.recv(length - len(data))
        if not packet:
            return None
        data += packet
    return data

# Send data in chunks(For Huge data => chunk by chunk)
def send_data(socket, data):
    data_pickle = pickle.dumps(data)
    data_size = len(data_pickle)
    socket.sendall(data_size.to_bytes(4, 'big'))  # Send data size first

    sent = 0
    while sent < data_size:
        chunk = data_pickle[sent:sent+4096]  # Send in chunks
        socket.sendall(chunk)
        sent += len(chunk)



#  Initialize server socket

In [3]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ('localhost', 1111)
server_socket.bind(server_address)
server_socket.listen(5)

# Set Federated Learning Server Parameters

In [4]:
# Maximum number of server_rounds or aggregation local models
server_rounds = 10   

client_num = 4
clients = []
client_params = {}

# Define Local Models Structure

In [5]:
num_features = 784
n0 = num_features
n1 = 20
n2 = 15
n3 = 10

# Initialize Client Local Models Paramters 

In [6]:
# Initialize weights
np.random.seed(1)
a = -1
b = 1
w1 = np.random.uniform(a, b, size=(n1, n0))
w2 = np.random.uniform(a, b, size=(n2, n1))
w3 = np.random.uniform(a, b, size=(n3, n2))

# Initialize biases
b1 = np.random.uniform(a, b, size=(n1, 1))
b2 = np.random.uniform(a, b, size=(n2, 1))
b3 = np.random.uniform(a, b, size=(n3, 1))


initial_parameters = {'weights': [w1, w2, w3], 'bias': [b1, b2, b3], 'server_rounds':server_rounds}

# Config For Save Train And Test Results 

In [7]:
# Initialize DataFrame to store
train_results_columns = ['Round', 'Client_ID', 'First_Layer_Weights', 'Second_Layer_Weights','Third_Layer_Weights', 'First_Layer_Bias', 'Second_Layer_Bias', 'Third_Layer_Bias','Train_MSE','Train_Accuracy']
train_results = pd.DataFrame(columns=train_results_columns)

server_model_columns= ['Round', 'First_Layer_Weights', 'Second_Layer_Weights','Third_Layer_Weights', 'First_Layer_Bias', 'Second_Layer_Bias', 'Third_Layer_Bias']
server_model = pd.DataFrame(columns=server_model_columns)

test_results_columns= ['Round', 'Client_ID','Test_MSE','Test_Accuracy']
test_results = pd.DataFrame(columns=test_results_columns)

# Federated Learning Process Handling On Server

In [8]:
print('**************************************************************************************************')
print('*********************************** WATTING FOR CONNECTIONS **************************************')
print('**************************************************************************************************')

while len(clients) < client_num: 
    client, client_address = server_socket.accept()
    clients.append(client)
    print(f"Connection from {client_address}")
    
    # Send initial params for current iteration to the clients
    send_data(client, initial_parameters)
    
print("Sent Initial Params To All Clients")

print("\n\033[1;m" + "*" * 125)

current_round = 0 
while current_round < server_rounds:
    # Send ready signal to clients
    for client in clients:
        client.send(pickle.dumps({'ready': True}))
    
    print("Start Round",current_round+1)
    ############################################################
    #  Wait For Local Model Train Results
    ############################################################
    # Receive updated train params from clients for the next round
    client_train_res = {}
    print('Pending Receive Params From Clients For Next Round')
    
    for client in clients:
        data_size = int.from_bytes(client.recv(4), 'big')  # Receive data size first
        data = receive_data(client, data_size) 
        client_train_res[client.getpeername()] = pickle.loads(data)
        print('Received From ClientID: ',client_train_res[client.getpeername()]["client_id"])
    
    
    
    ############################################################
    #  Aggregate weights based on the number of samples
    ############################################################
    
    print("Start Aggregation Process")
    total_samples = sum(client_train_res[addr]['num_samples'] for addr in client_train_res)
    # Calculate weighted average of params from clients
    aggregated_parameters = {
        'weights': [np.zeros_like(w1), np.zeros_like(w2), np.zeros_like(w3)],
        'bias': [np.zeros_like(b1), np.zeros_like(b2), np.zeros_like(b3)],
        'current_server_round':current_round
        }
    
    for addr, client_train_data in client_train_res.items():
        ratio = client_train_data['num_samples'] / total_samples 
        for i in range(len(aggregated_parameters['weights'])): 
            aggregated_parameters['weights'][i] += client_train_data['weights'][i] * ratio
            aggregated_parameters['bias'][i] += client_train_data['bias'][i] * ratio
    
    print("End Aggregation Process")
    
    
    
    ############################################################
    #  Send updated model to clients for evaluate by test data
    ############################################################
    for client in clients:
        send_data(client, aggregated_parameters)
    print("Sent Aggregated Model Params To Client To Evaluate On Test Data")   
    
    print('Pending Receive Test results From Clients')
    # Receive evaluate global model on test-result from clients 
    client_test_res = {}
    for client in clients: 
        data_size = int.from_bytes(client.recv(4), 'big')  # Receive data size first
        data = receive_data(client, data_size)
        client_test_res[client.getpeername()] = pickle.loads(data)
        print('Received From ClientID: ',client_test_res[client.getpeername()]["client_id"])
    
    
    # Save train results for each client in each round on a excel file
    for addr, client_train_data in client_train_res.items(): 
        # Convert weights matrices to JSON format
        first_layer_weights_json = json.dumps({f"w{row+1}_{col+1}": client_train_data['weights'][0][row][col]
                                              for row in range(client_train_data['weights'][0].shape[0])
                                              for col in range(client_train_data['weights'][0].shape[1])})
        
        second_layer_weights_json = json.dumps({f"w{row+1}_{col+1}": client_train_data['weights'][1][row][col]
                                               for row in range(client_train_data['weights'][1].shape[0])
                                               for col in range(client_train_data['weights'][1].shape[1])})
        
        third_layer_weights_json = json.dumps({f"w{row+1}_{col+1}": client_train_data['weights'][2][row][col]
                                              for row in range(client_train_data['weights'][2].shape[0])
                                              for col in range(client_train_data['weights'][2].shape[1])})
        
        first_layer_bias_json = json.dumps({f"b{row+1}_{col+1}": client_train_data['bias'][0][row][col]
                                              for row in range(client_train_data['bias'][0].shape[0])
                                              for col in range(client_train_data['bias'][0].shape[1])})
        
        second_layer_bias_json = json.dumps({f"b{row+1}_{col+1}": client_train_data['bias'][1][row][col]
                                               for row in range(client_train_data['bias'][1].shape[0])
                                               for col in range(client_train_data['bias'][1].shape[1])})
        
        third_layer_bias_json = json.dumps({f"b{row+1}_{col+1}": client_train_data['bias'][2][row][col]
                                              for row in range(client_train_data['bias'][2].shape[0])
                                              for col in range(client_train_data['bias'][2].shape[1])})

        train_results = train_results.append({
            'Round': current_round + 1,
            'Client_ID': client_train_data["client_id"],
            'First_Layer_Weights': first_layer_weights_json,
            'Second_Layer_Weights': second_layer_weights_json,
            'Third_Layer_Weights': third_layer_weights_json,
            'First_Layer_Bias': first_layer_bias_json,
            'Second_Layer_Bias': second_layer_bias_json,
            'Third_Layer_Bias': third_layer_bias_json,
            'Train_MSE': client_train_data["Train_MSE"],
            'Train_Accuracy': client_train_data["Train_Accuracy"]*100
        }, ignore_index=True)
        
         
    # save aggregated model params
    # Convert aggregated matrices to JSON format
    first_layer_weights_json = json.dumps({f"w{row+1}_{col+1}": aggregated_parameters['weights'][0][row][col]
                                            for row in range(aggregated_parameters['weights'][0].shape[0])
                                            for col in range(aggregated_parameters['weights'][0].shape[1])})
        
    second_layer_weights_json = json.dumps({f"w{row+1}_{col+1}": aggregated_parameters['weights'][1][row][col]
                                            for row in range(aggregated_parameters['weights'][1].shape[0])
                                            for col in range(aggregated_parameters['weights'][1].shape[1])})
        
    third_layer_weights_json = json.dumps({f"w{row+1}_{col+1}": aggregated_parameters['weights'][2][row][col]
                                            for row in range(aggregated_parameters['weights'][2].shape[0])
                                            for col in range(aggregated_parameters['weights'][2].shape[1])})
        
    first_layer_bias_json = json.dumps({f"b{row+1}_{col+1}": aggregated_parameters['bias'][0][row][col]
                                            for row in range(aggregated_parameters['bias'][0].shape[0])
                                            for col in range(aggregated_parameters['bias'][0].shape[1])})
        
    second_layer_bias_json = json.dumps({f"b{row+1}_{col+1}": aggregated_parameters['bias'][1][row][col]
                                            for row in range(aggregated_parameters['bias'][1].shape[0])
                                            for col in range(aggregated_parameters['bias'][1].shape[1])})
        
    third_layer_bias_json = json.dumps({f"b{row+1}_{col+1}": aggregated_parameters['bias'][2][row][col]
                                            for row in range(aggregated_parameters['bias'][2].shape[0])
                                            for col in range(aggregated_parameters['bias'][2].shape[1])})
        
    server_model = server_model.append({
            'Round': current_round + 1,
            'First_Layer_Weights': first_layer_weights_json,
            'Second_Layer_Weights': second_layer_weights_json,
            'Third_Layer_Weights': third_layer_weights_json,
            'First_Layer_Bias': first_layer_bias_json,
            'Second_Layer_Bias': second_layer_bias_json,
            'Third_Layer_Bias': third_layer_bias_json
        }, ignore_index=True)
    
    
    # Save test results for each client
    for addr, client_test_data in client_test_res.items(): 
        
        test_results = test_results.append({ 
            'Round': current_round + 1,
            'Client_ID': client_test_data["client_id"], 
            'Test_MSE': client_test_data["Test_MSE"],
            'Test_Accuracy':client_test_data["Test_Accuracy"]*100
        }, ignore_index=True)
  

    print("End Round",current_round+1)
    print("\n\033[1;m" + "*" * 125)
    
    current_round += 1
    
# Send termination signal to clients
for client in clients:
    client.send(pickle.dumps({'terminate': True}))

# Export DataFrames to Excel file
train_results.to_excel('train_results.xlsx', index=False) 
server_model.to_excel('server_model.xlsx', index=False)
test_results.to_excel('test_results.xlsx', index=False)

# Close connections
server_socket.close()

print('**************************************************************************************************')
print('******************************************* FL END ***********************************************')
print('**************************************************************************************************')

**************************************************************************************************
*********************************** WATTING FOR CONNECTIONS **************************************
**************************************************************************************************
Connection from ('127.0.0.1', 14417)
Connection from ('127.0.0.1', 14418)
Connection from ('127.0.0.1', 14428)
Connection from ('127.0.0.1', 14461)
Sent Initial Params To All Clients

*****************************************************************************************************************************
Start Round 1
Pending Receive Params From Clients For Next Round
Received From ClientID:  C1
Received From ClientID:  C3
Received From ClientID:  C2
Received From ClientID:  C4
Start Aggregation Process
End Aggregation Process
Sent Aggregated Model Params To Client To Evaluate On Test Data
Pending Receive Test results From Clients
Received From ClientID:  C1
Received From ClientID:  C3
Rece

In [9]:
server_model.to_excel('server_model.xlsx', index=False)
